In [ ]:
import numpy as np
from pathlib import Path
dest_path='data/EMP'
dataset_name='H3_txt'
split='test'

base_path = Path(dest_path) / dataset_name / split

all_seqs = []
all_labels = []
label_mapper = {}

for i, x in enumerate(base_path.iterdir()):
    label_mapper[x.stem] = i

for label_type in label_mapper.keys():
    for path in (base_path / label_type).iterdir():
        with open(path, "r") as f:
            content = f.read()
        all_seqs.append(content)
        all_labels.append(label_mapper[label_type])

In [ ]:
#import autotokenizer
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn
from sklearn.metrics import matthews_corrcoef as mcc
max_length=128
restrict = lambda x: (
                    torch.cumsum(x, dim=-2)
                    / torch.arange(
                        1, 1 + x.size(-2), device=x.device, dtype=x.dtype
                    ).unsqueeze(-1)
                )[..., -1:, :]          
with torch.no_grad():
    bert2_tokenizer=AutoTokenizer.from_pretrained("hyena-dna/weight/dnabert2",trust_remote_code=True)
    bert2_model=AutoModel.from_pretrained("hyena-dna/weight/dnabert2",trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('hyena-dna/outputs/2024-05-05/09-15-14-919144/checkpoints/val/mcc.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    bert2_decoder = nn.Linear(768,2).to('cuda')
    #edit key name in hyena_decoder
    state_dict=bert2_decoder.state_dict()
    
    bert2_model.load_state_dict(checkpoint,strict=False)
    bert2_decoder.load_state_dict(checkpoint,strict=False)
    bert2_model.eval()
    bert2_decoder.eval()
    out1_list=[]
    label_list=[]
    for i in range(len(all_seqs)):
        sequence_encoded=bert2_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )


        sequence_tensor=torch.tensor(sequence_encoded['input_ids']).to('cuda')
        sequence_tensor=torch.reshape(sequence_tensor,(1,max_length))
        hidden_states=bert2_model(input_ids=sequence_tensor,export_hidden_states=True)[0]
        hidden_states=restrict(hidden_states)
        out1=bert2_decoder(hidden_states)
        out1=torch.argmax(out1,dim=-1)
        out1_list.append(out1.item())
        label_list.append(all_labels[i])
        print(mcc(out1_list,label_list))


    #calculate the 
    
